# TODO

2. Document execute_model function
4. Format outputs from model
 

In [1]:
# Refactoring NASEM model
import nasem_dairy as nd
import pandas as pd
import math
from pprint import pprint
from icecream import ic
# use package dir to always use /data folder regardless of where this is executed from
import importlib_resources

path_to_package_data = importlib_resources.files("nasem_dairy.data")

# Read_csv to load required data into env
user_diet_in, animal_input_in, equation_selection_in = nd.read_csv_input(path_to_package_data.joinpath("./input.csv"))

# Load feed library
feed_library_in = pd.read_csv(path_to_package_data.joinpath("NASEM_feed_library.csv"))

# Prepare infusion data (This will be optional as a default dict of 0 is provided to function otherwise)
infusion_custom = nd.read_infusion_input(path_to_package_data.joinpath("infusion_input.csv"))


# Execute Model

In [8]:
animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)


# Run model - different options

In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)

# diet_info
An_data

## Toggle infusions
Notes: found that infusions are being imported within the function, need to move all file imports out of the run model function.
There is also double up of the logic for use_infusions, so modifying that to check which one to use once. 
It could actually be avoided by just providing a default infusion dictionary if one is not given to function directly.
These changes make the 'use_infusions' flag obsolete

In [5]:
# Test DMI equations
equation_selection_in['use_infusions'] = 0 # FALSE

animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)

# diet_info
# infusion_data


In [6]:
# equation_selection_in['use_infusions'] = 1

animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict,
    infusion_input= infusion_custom)

# diet_info
model_out_dict

{'infusion_data': {'Inf_Acet_g': 3000.0,
  'Inf_ADF_g': 4000.0,
  'Inf_Arg_g': 5000.0,
  'Inf_Ash_g': 6000.0,
  'Inf_Butr_g': 7000.0,
  'Inf_CP_g': 8000.0,
  'Inf_CPARum_CP': 9000.0,
  'Inf_CPBRum_CP': 10000.0,
  'Inf_CPCRum_CP': 11000.0,
  'Inf_dcFA': 12000.0,
  'Inf_dcRUP': 13000.0,
  'Inf_DM_g': 14000.0,
  'Inf_EE_g': 15000.0,
  'Inf_FA_g': 16000.0,
  'Inf_Glc_g': 17000.0,
  'Inf_His_g': 18000.0,
  'Inf_Ile_g': 19000.0,
  'Inf_KdCPB': 20000.0,
  'Inf_Leu_g': 21000.0,
  'Inf_Lys_g': 22000.0,
  'Inf_Met_g': 23000.0,
  'Inf_NDF_g': 24000.0,
  'Inf_NPNCP_g': 25000.0,
  'Inf_Phe_g': 26000.0,
  'Inf_Prop_g': 27000.0,
  'Inf_St_g': 28000.0,
  'Inf_Thr_g': 29000.0,
  'Inf_Trp_g': 30000.0,
  'Inf_ttdcSt': 31000.0,
  'Inf_Val_g': 32000.0,
  'Inf_VFA_g': 33000.0,
  'Inf_Location': nan,
  'Inf_DMIn': 14.0,
  'Inf_StIn': 28.0,
  'Inf_NDFIn': 24.0,
  'Inf_ADFIn': 4.0,
  'Inf_GlcIn': 17.0,
  'Inf_CPIn': 8.0,
  'Inf_NPNCPIn': 25.0,
  'Inf_FAIn': 16.0,
  'Inf_AshIn': 6.0,
  'Inf_VFAIn': 33.0,
  'Inf

## Test DMI equation selections

In [7]:
# Equation = 0 should match user input:
equation_selection_in['DMIn_eqn'] = 0
animal_input_in['DMI'] = 24.521 # same as .csv file

animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)

animal_input['DMI']

24.521

In [8]:
# Equation = 8 should calculate calculate_Dt_DMIn_Lact1()
equation_selection_in['DMIn_eqn'] = 8
animal_input_in['DMI'] = 24.521 # same as .csv file

animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)

animal_input['DMI']

21.687577670699824

In [9]:
# make function to test all values
def iterate_test_DMI_eqns(DMIn_eqn):
    equation_selection_in_test = equation_selection_in.copy()
    equation_selection_in_test['DMIn_eqn'] = DMIn_eqn

    animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
        user_diet = user_diet_in, 
        animal_input = animal_input_in, 
        equation_selection = equation_selection_in_test, 
        feed_library_df = feed_library_in, 
        coeff_dict = nd.coeff_dict)

    return (animal_input['DMI'])

[iterate_test_DMI_eqns(DMIn_eqn_value) for DMIn_eqn_value in range(0, 16)]


DMIn_eqn uncaught - DMI not changed. equation_selection[DMIn_eqn]: 1
DMIn_eqn uncaught - DMI not changed. equation_selection[DMIn_eqn]: 9


[24.521,
 24.521,
 11.504513756424398,
 16.842879099441184,
 11.474631902511609,
 12.805095198359778,
 10.866105707801568,
 12.251816154227694,
 21.687577670699824,
 24.521,
 12.323872879290622,
 12.364693050000001,
 11.504513756424398,
 16.842879099441184,
 11.474631902511609,
 12.805095198359778]

In [ ]:
## Still need to test this condition: 
#if animal_input['An_PrePartWk'] > An_PrePartWkDurat:

print(animal_input['An_PrePartWk'])

# Calculate additional physiology values
An_PrePartDay = animal_input_in['An_GestDay'] - animal_input_in['An_GestLength']
An_PrePartDay / 7
# animal_input_in['An_PrePartWk']

# Test equation_selection['MiN_eqn']

In [10]:
# equation_selection['MiN_eqn']


# make function to test all values
def iterate_test_Du_MiN_g_eqns(MiN_eqn):
    print(MiN_eqn)
    equation_selection_in_test = equation_selection_in.copy()
    equation_selection_in_test['MiN_eqn'] = MiN_eqn

    animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
        user_diet = user_diet_in, 
        animal_input = animal_input_in, 
        equation_selection = equation_selection_in_test, 
        feed_library_df = feed_library_in, 
        coeff_dict = nd.coeff_dict)

    return (model_out_dict['Du_MiN_g'])

[iterate_test_Du_MiN_g_eqns(DMIn_eqn_value) for DMIn_eqn_value in range(1, 4)]

1
2
3


[299.31825131136395, 122.74827303321794, 276.5276811382165]

In [11]:
equation_selection_in

{'Use_DNDF_IV': 0.0,
 'DMIn_eqn': 8,
 'mProd_eqn': 0.0,
 'MiN_eqn': 1.0,
 'use_infusions': 0,
 'NonMilkCP_ClfLiq': 0.0,
 'Monensin_eqn': 0.0,
 'mPrt_eqn': 0.0}

# test Monensin eqn

In [12]:
equation_selection_in


{'Use_DNDF_IV': 0.0,
 'DMIn_eqn': 8,
 'mProd_eqn': 0.0,
 'MiN_eqn': 1.0,
 'use_infusions': 0,
 'NonMilkCP_ClfLiq': 0.0,
 'Monensin_eqn': 0.0,
 'mPrt_eqn': 0.0}

In [13]:
# make function to test all values
def iterate_test_Mon_eqns(Monensin_eqn):
    print(Monensin_eqn)
    equation_selection_in_test = equation_selection_in.copy()
    equation_selection_in_test['Monensin_eqn'] = Monensin_eqn

    animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
        user_diet = user_diet_in, 
        animal_input = animal_input_in, 
        equation_selection = equation_selection_in_test, 
        feed_library_df = feed_library_in, 
        coeff_dict = nd.coeff_dict)

    return (model_out_dict['Dt_DEIn'], model_out_dict['An_DEIn'])

[iterate_test_Mon_eqns(Monensin_eqn) for Monensin_eqn in range(0, 2)]

0
1


[(array(69.04770702), array(69.04770702)),
 (array(70.42866116), array(70.42866116))]